<a href="https://colab.research.google.com/github/lacteavian/miuul_machine_learning_bootcamp/blob/main/Potential_Customer_Return_with_Rule_Based_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A game company wants to create
level-based new customer definitions (personas) by using some characteristics of its customers, create segments according to these new customer definitions and estimate how much new customers can earn the company on average according to these segments.



For example: It is desired to determine how much a 25-year-old male IOS user from Turkey can earn on average.


# task 1

Read the file persona.csv and show general information about the data set.

In [102]:
import pandas as pd

In [103]:
df = pd.read_csv("/content/persona.csv")

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


We have 2 numerical and 3 categorical attributes.

In [105]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PRICE,5000.0,34.1320,12.464897,9.0,29.0,39.0,39.0,59.0
AGE,5000.0,23.5814,8.995908,15.0,17.0,21.0,27.0,66.0


we can only analyse numeric variables.

In [106]:
df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            4990, 4991, 4992, 4993, 4994, 4995, 4996, 4997, 4998, 4999],
           dtype='int64', length=5000)

In [107]:
df.shape

(5000, 5)

In [108]:
df.isnull().values.any()

False

We can see there is no empty value at all

In [109]:
df.columns

Index(['PRICE', 'SOURCE', 'SEX', 'COUNTRY', 'AGE'], dtype='object')

In [110]:
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [111]:
df.tail()

,PRICE,SOURCE,SEX,COUNTRY,AGE
4995,29,android,female,bra,31
4996,29,android,female,bra,31
4997,29,android,female,bra,31
4998,39,android,female,bra,31
4999,29,android,female,bra,31


How many unique SOURCEs are there? What are their frequencies?

In [112]:
df["SOURCE"].unique()

array(['android', 'ios'], dtype=object)

In [113]:
df["SOURCE"].value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

How many unique PRICEs are there?

In [114]:
df["PRICE"].nunique()

6

In [115]:
df["PRICE"].value_counts()

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

How many sales were there from which country?


In [116]:
df["COUNTRY"].value_counts()

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

how much was earned from sales by country

In [117]:
df.groupby("COUNTRY").agg({"PRICE": "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


what are the sales numbers by SOURCE types?

In [118]:
df["SOURCE"].value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

What are the PRICE averages by COUNTRY?

In [119]:
df.groupby("COUNTRY").agg({"PRICE": "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


What are the PRICE averages by SOURCE?

In [120]:
df.groupby("SOURCE").agg({"PRICE": "mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


What are the PRICE averages in the COUNTRY-SOURCE breakdown?


In [121]:
df.groupby(["SOURCE","COUNTRY"]).agg({"PRICE": "mean"})

PRICE
SOURCE  COUNTRY           
android bra      34.387029
        can      33.330709
        deu      33.869888
        fra      34.312500
        tur      36.229437
        usa      33.760357
ios     bra      34.222222
        can      33.951456
        deu      34.268817
        fra      32.776224
        tur      33.272727
        usa      34.371703

# TASK 2: What are the average earnings in breakdown of COUNTRY, SOURCE, SEX, AGE?

In [122]:
pd.set_option("display.max_rows",5000)

In [123]:
new_df = df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg({"PRICE": "mean"})

# TASK 3: Sort the output by PRICE?

we apply the sort_values method to PRICE in descending order to see the output in the previous question better.

In [124]:
new_df = new_df.sort_values("PRICE", ascending=False)


In [125]:
new_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [126]:
len(new_df.columns)

1

# TASK 4: Convert the names in the index to variable names.

In [127]:
new_df = new_df.reset_index()

In [128]:
new_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


In [129]:
new_df.columns

Index(['COUNTRY', 'SOURCE', 'SEX', 'AGE', 'PRICE'], dtype='object')

# TASK 5: Convert AGE variable to categorical variable and add it to agg_df.

In [130]:
bins = [0, 18, 23, 30, 40, new_df["AGE"].max()]

In [131]:
labels = ["0_18","19_23","24_30","31_40", f'41_{new_df["AGE"].max()}']

In [132]:
new_df["AGE_CAT"] = pd.cut(new_df["AGE"], bins=bins, labels=labels)

In [133]:
new_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


# TASK 6: Identify new level based customers and add them as variables to the dataset.

In [134]:
new_df["CUSTOMERS_LEVEL_BASED"] = [
    "_".join(i).upper() for i in new_df.drop(["AGE", "PRICE"], axis=1).values]


In [135]:
new_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,CUSTOMERS_LEVEL_BASED
0,bra,android,male,46,59.0,41_66,BRA_ANDROID_MALE_41_66
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40


In [136]:
new_df = new_df[["CUSTOMERS_LEVEL_BASED", "PRICE"]]


In [137]:
new_df = new_df.groupby("CUSTOMERS_LEVEL_BASED")["PRICE"].mean().reset_index()


In [138]:
new_df.head()

,CUSTOMERS_LEVEL_BASED,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41_66,36.737179


# TASK 7: Segment new customers (USA_ANDROID_MALE_0_18).

In [139]:
new_df["SEGMENT"] = pd.qcut(new_df.PRICE, q=4, labels=["D", "C", "B","A"])


In [140]:
new_df.head()

,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,35.645303,B
1,BRA_ANDROID_FEMALE_19_23,34.077340,C
2,BRA_ANDROID_FEMALE_24_30,33.863946,C
3,BRA_ANDROID_FEMALE_31_40,34.898326,B
4,BRA_ANDROID_FEMALE_41_66,36.737179,A


In [141]:
new_df.groupby("SEGMENT").agg({"PRICE": "mean"}).reset_index()


,SEGMENT,PRICE
0,D,29.206780
1,C,33.509674
2,B,34.999645
3,A,38.691234


In [142]:
new_df.head()

,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,35.645303,B
1,BRA_ANDROID_FEMALE_19_23,34.077340,C
2,BRA_ANDROID_FEMALE_24_30,33.863946,C
3,BRA_ANDROID_FEMALE_31_40,34.898326,B
4,BRA_ANDROID_FEMALE_41_66,36.737179,A


# TASK 8: Classify the new customers and estimate how much income they can bring.

In [143]:
new_user = "TUR_ANDROID_FEMALE_31_40"


In [144]:
new_df[new_df["CUSTOMERS_LEVEL_BASED"] == new_user]


,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
72,TUR_ANDROID_FEMALE_31_40,41.833333,A


In [145]:
new_user = "FRA_IOS_FEMALE_31_40"


In [146]:
new_df[new_df["CUSTOMERS_LEVEL_BASED"] == new_user]


,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
63,FRA_IOS_FEMALE_31_40,32.818182,C


In [147]:
new_df[new_df["CUSTOMERS_LEVEL_BASED"] == "BRA_ANDROID_FEMALE_0_18"]


,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,35.645303,B


In [148]:
df[["PRICE","AGE"]].corr()

,PRICE,AGE
PRICE,1.000000,-0.010202
AGE,-0.010202,1.000000
